# Car_evaluation_classification

## import Libraries

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Data Load & Preprocessing

In [51]:
df = pd.read_csv("C:/Users/plo55/car_evaluation.csv", header=None) # index_col = 0..?, # CSV 파일의 첫 번째 행(header)을 컬럼 이름으로 사용하지 않음
# 파일에 헤더(컬럼 이름)가 없는 상태에서 header=None을 생략하면, 첫 번째 행이 컬럼 이름으로 사용됨(원래 데이터인데 헤더로 변함!).
# 그래서 컬럼명을 직접 설정하려면 header=None을 넣어야 함!
# index_col=0이면 첫 번째 열을 인덱스로 지정한다는 뜻이다. 하지만 car_evaluation.csv는 첫 번째 열이 데이터 값이므로 index_col=0을 사용하지 않는 게 맞다.
print(df.head())
print("\n=====================================================\n")

df.columns = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety', 'output'] # 각 컬럼명을 직접 지정
# price: 가격, maint: 유지비, doors: 문 개수, persons: 수용 인원, lug_capacity: 트렁크 크기, safety: 안전성, output: 자동차 평가 결과 (label)
print(df.columns) # 컬럼 확인 # print(df['output']) 특정 컬럼 확인
print("\n=====================================================\n")

# 결측치 확인
print(df.isnull().sum())
print("\n=====================================================\n")

# 레이블 불균형 여부를 위한 갯수 확인 (숫자)
print(df['output'].value_counts())
print("\n=====================================================\n")
# output 불균형이 심한 것을 확인할 수 있다.
# unacc    1210
# acc       384
# good       69
# vgood      65
# 이 경우 SMOTE를 이용해서 데이터를 증강하는 방법을 사용할 수 있다. 데이터가 적은 good과 vgood 클래스를 인위적으로 늘려서 학습을 해결하는 것.


# 범주형 데이터를 숫자로 변환
# 머신러닝 모델은 숫자 데이터를 입력으로 받기 때문에 범주형 데이터를 숫자로 변환해야 함.
label_encoders = {} # 각 컬럼별 LabelEncoder 객체를 저장할 딕셔너리 생성
columns = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety', 'output']
for column in columns:
  label_encoders[column] = LabelEncoder() # 해당 컬럼에 대해 LabelEncoder 객체 생성
  df[column] = label_encoders[column].fit_transform(df[column]) # 데이터를 숫자로 변환

# 레이블 갯수 확인 (숫자)
print(df['output'].value_counts())
print("\n=====================================================\n")

       0      1  2  3      4     5      6
0  vhigh  vhigh  2  2  small   low  unacc
1  vhigh  vhigh  2  2  small   med  unacc
2  vhigh  vhigh  2  2  small  high  unacc
3  vhigh  vhigh  2  2    med   low  unacc
4  vhigh  vhigh  2  2    med   med  unacc


Index(['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety',
       'output'],
      dtype='object')


price           0
maint           0
doors           0
persons         0
lug_capacity    0
safety          0
output          0
dtype: int64


output
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64


output
2    1210
0     384
1      69
3      65
Name: count, dtype: int64




## Data Preparation & Scaling

In [54]:
# .values: 데이터프레임을 넘파이 배열로 변경함.
X = df.drop('output', axis=1).values # Feature
Y = df['output'].values # Label
# DataFrame와 numpy 차이점
# - numpy는 배열이므로 컬럼이 없음. 연산 속도가 더 빠른 대신 조작이 불편함.
# - 반면에 df는 컬럼명이 있어서 데이터 조작이 편하지만 연산 속도가 느림.
# - 그래서 데이터 분석 및 전처리를 df상태에서 하고 마지막에 numpy형식으로 변환하는 것.
# 굳이 넘파이 배열로 바꾸어 주는 것을 명시하지 않아도 작동하기는 함!

print(X)
print(Y)

# 학습 데이터 & 테스트 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 분할된 데이터의 shape을 출력
print("\n=====================================================\n")
print(X_train.shape) # (1382, 6) 6열짜리 데이터가 1382행 있다는 뜻.
print(X_test.shape)
print(Y_train.shape) # (1382, ) 1열짜리 데이터가 1382행 있다는 뜻.
print(Y_test.shape) 

[[3 3 0 0 2 1]
 [3 3 0 0 2 2]
 [3 3 0 0 2 0]
 ...
 [1 1 3 2 0 1]
 [1 1 3 2 0 2]
 [1 1 3 2 0 0]]
[2 2 2 ... 2 1 3]


(1382, 6)
(346, 6)
(1382,)
(346,)


## Train & Test & Report

In [57]:
# Decision Tree (DT)
print("\n=====================================================\n")
dt_model = DecisionTreeClassifier() # DT 모델 생성.
dt_model.fit(X_train, Y_train) # 모델 학습.
dt_predictions = dt_model.predict(X_test) # 예측. 즉, dt_predictions에는 테스트 데이터 특성(X_test)에 따라 예측한 Y값이 들어있는 것.
print("Decision Tree Accuracy:", accuracy_score(Y_test, dt_predictions)) # 예측한 값과 실제 Y값을 비교하여 정확도 평가.
print("\nDecision Tree Classification Report:\n", classification_report(Y_test, dt_predictions, zero_division=1))
print("DT Confusion Matrix:\n", confusion_matrix(Y_test, dt_predictions)) # 혼동 행렬 출력

# Random Forest (RF)
print("\n=====================================================\n")
rf_model = RandomForestClassifier() # RF 모델 생성.
rf_model.fit(X_train, Y_train) # 모델 학습.
rf_predictions = rf_model.predict(X_test) # 예측. 즉, rf_predictions에는 테스트 데이터 특성(X_test)에 따라 예측한 Y값이 들어있는 것.
print("Random Forest Accuracy:", accuracy_score(Y_test, rf_predictions)) # 예측한 값과 실제 Y값을 비교하여 정확도 평가.
print("\nRandom Forest Classification Report:\n", classification_report(Y_test, rf_predictions, zero_division=1))
print("RF Confusion Matrix:\n", confusion_matrix(Y_test, rf_predictions)) # 혼동 행렬 출력

# Support Vector Machine (SVM)
print("\n=====================================================\n")
svm_model = SVC() # SVM 모델 생성.
svm_model.fit(X_train, Y_train) # 모델 학습.
svm_predictions = svm_model.predict(X_test) # 예측. 즉, svm_predictions에는 테스트 데이터 특성(X_test)에 따라 예측한 Y값이 들어있는 것.
print("SVM Accuracy:", accuracy_score(Y_test, svm_predictions)) # 예측한 값과 실제 Y값을 비교하여 정확도 평가.
print("\nSVM Classification Report:\n", classification_report(Y_test, svm_predictions, zero_division=1))
print("SVM Confusion Matrix:\n", confusion_matrix(Y_test, svm_predictions)) # 혼동 행렬 출력

# Logistic Regression (LR)
print("\n=====================================================\n")
lr_model = LogisticRegression() # LR 모델 생성.
lr_model.fit(X_train, Y_train) # 모델 학습.
lr_predictions = lr_model.predict(X_test) # 예측. 즉, lr_predictions에는 테스트 데이터 특성(X_test)에 따라 예측한 Y값이 들어있는 것.
print("Logistic Regression Accuracy:", accuracy_score(Y_test, lr_predictions)) # 예측한 값과 실제 Y값을 비교하여 정확도 평가.
print("\nLogistic Regression Classification Report:\n", classification_report(Y_test, lr_predictions, zero_division=1))
print("LR Confusion Matrix:\n", confusion_matrix(Y_test, lr_predictions)) # 혼동 행렬 출력
# Logistic Regression은 클래스 불균형과 비선형 문제에 약한데, Car_evaluation의 경우 레이블 불균형이 심해서 모델 성능(예측 성공률)이 낮게 나온다.

# K-Nearest Neighbors (KNN)
print("\n=====================================================\n")
knn_model = KNeighborsClassifier(n_neighbors=5)  # 기본적으로 K=5 사용
knn_model.fit(X_train, Y_train)
knn_predictions = knn_model.predict(X_test)
print("KNN Accuracy:", accuracy_score(Y_test, knn_predictions))
print("\nKNN Algorithm Classification Report:\n", classification_report(Y_test, knn_predictions, zero_division=1))
print("KNN Confusion Matrix:\n", confusion_matrix(Y_test, knn_predictions)) # 혼동 행렬 출력



Decision Tree Accuracy: 0.9682080924855492

Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.92      0.94        83
           1       0.56      0.91      0.69        11
           2       1.00      1.00      1.00       235
           3       1.00      0.82      0.90        17

    accuracy                           0.97       346
   macro avg       0.88      0.91      0.88       346
weighted avg       0.98      0.97      0.97       346

DT Confusion Matrix:
 [[ 76   6   1   0]
 [  1  10   0   0]
 [  0   0 235   0]
 [  1   2   0  14]]


Random Forest Accuracy: 0.9682080924855492

Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.89      0.94        83
           1       0.56      0.91      0.69        11
           2       1.00      1.00      1.00       235
           3       0.94      0.94      0.94        17

    accuracy              

## 혼동 행렬에 대하여

혼동행렬
- 실질적으로 어떤 클래스가 잘 예측됐는지 안됐는지 디테일하게 알기 위해서 사용.
- 모델이 얼마나 정확하게 각 클래스를 예측했는지 시각적으로 확인할 수 있는 도구.
  
혼동행렬은 분류 모델의 성능을 평가하는 데 사용되는 표로, 실제 값과 모델이 예측한 값의 관계를 시각적으로 나타낸다. 각 행은 실제 클래스(진짜 클래스)를 나타내고, 각 열은 예측된 클래스(모델의 예측)를 나타낸다.

혼동행렬은 보통 다음과 같은 형태로 나타낸다:

|               | Predicted Class 0 | Predicted Class 1 | Predicted Class 2 | Predicted Class 3 |
|---------------|--------------------|--------------------|--------------------|--------------------|
| **True Class 0** | True Positive (TP)   | False Positive (FP)  | False Negative (FN) | False Negative (FN) |
| **True Class 1** | False Positive (FP)  | True Positive (TP)   | False Negative (FN) | False Negative (FN) |
| **True Class 2** | False Negative (FN)  | False Negative (FN)  | True Positive (TP)  | False Positive (FP) |
| **True Class 3** | False Negative (FN)  | False Negative (FN)  | False Negative (FN) | True Positive (TP)  |

예시
SVM Confusion Matrix:
\
\begin{bmatrix}
    68 & 5  & 10 & 0 \\ 
    6  & 4  & 0  & 1 \\
    10 & 0  & 225 & 0 \\
    2  & 0  & 0  & 15
\end{bmatrix}

 첫번째 행: 실제 클래스 0에 대해 예측한 클래스들<br>
  - 68: 실제 클래스 0인 데이터가 모델에 의해 정확히 클래스 0으로 예측된 수 (True Positive)
  - 5: 실제 클래스 0인 데이터가 모델에 의해 잘못 클래스 1로 예측된 수 (False Positive)
  - 10: 실제 클래스 0인 데이터가 모델에 의해 잘못 클래스 2로 예측된 수 (False Positive)
  - 0: 실제 클래스 0인 데이터가 모델에 의해 잘못 클래스 3으로 예측된 수 (False Positive)<br>

 두번째 행: 실제 클래스 1에 대해 예측한 클래스들<br>
  - 6: 실제 클래스 1인 데이터가 모델에 의해 잘못 클래스 0으로 예측된 수 (False Negative)
  - 4: 실제 클래스 1인 데이터가 모델에 의해 정확히 클래스 1로 예측된 수 (True Positive)
  - 0: 실제 클래스 1인 데이터가 모델에 의해 잘못 클래스 2로 예측된 수 (False Positive)
  - 1: 실제 클래스 1인 데이터가 모델에 의해 잘못 클래스 3으로 예측된 수 (False Positive)
<br>

 세번째 행: 실제 클래스 2에 대해 예측한 클래스들<br>
 네번째 행: 실제 클래스 3에 대해 예측한 클래스들<br>
 
여기서:
- **True Positive (TP)**: 실제 양성 클래스이면서, 모델이 양성 클래스라고 예측한 경우
- **False Positive (FP)**: 실제 음성 클래스이지만, 모델이 양성 클래스라고 잘못 예측한 경우
- **False Negative (FN)**: 실제 양성 클래스이지만, 모델이 음성 클래스라고 잘못 예측한 경우
- **True Negative (TN)**: 실제 음성 클래스이면서, 모델이 음성 클래스라고 예측한 경우

## 혼동행렬의 지표

혼동행렬을 통해 다양한 성능 지표를 계산할 수 있다. 대표적인 지표는 다음과 같다:

- **정확도 (Accuracy)**: 모델이 얼마나 정확하게 예측했는지. Accuracy = (TP + TN) / (전체 데이터)
- **정밀도 (Precision)**: 특정 클래스에 대해 얼마나 정확하게 예측했는지. Precision = TP / (TP + FP)
- **재현율 (Recall)**: 실제 특정 클래스가 얼마나 잘 예측되었는지. Recall = TP / (TP + FN)
- **F1 Score**: 정밀도와 재현율의 조화 평균. F1 Score = 2 * (Precision * Recall) / (Precision + Recall)


